# Imports

## Dependencies

In [2]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from io import BytesIO
import datetime
import numpy as np
import configparser
import sys

## Micromet

In [3]:
#sys.path.append("G:/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/MicroMet")
#sys.path.append("C:/Users/kladig/Documents/GitHub/MicroMet")
sys.path.append("../..")
import micromet

## Connection info and Config

In [4]:
config = configparser.ConfigParser()

config.read('../../secrets/config.ini')

from sqlalchemy import create_engine
import urllib.parse
host = config['DEFAULT']['ip']
pw = config['DEFAULT']['pw']
user = config['DEFAULT']['login']

encoded_password = urllib.parse.quote_plus(pw)

def postconn_et(encoded_password, host='localhost',user='postgres',port='5432',db='groundwater', schema = 'groundwater'):
    connection_text = "postgresql+psycopg2://{:}:{:}@{:}:{:}/{:}?gssencmode=disable".format(user,encoded_password,host,port,db)
    return create_engine(connection_text, connect_args={'options': '-csearch_path={}'.format(schema)})


engine = postconn_et(encoded_password, host=host, user=user)

# Pull From Each Datalogger Using the CS Datalogger API

https://help.campbellsci.com/crbasic/cr1000x/Content/Info/webserverapicommands1.htm

In [18]:
station = 'US-UTJ'
sql = f"SELECT * FROM amfluxeddy WHERE stationid = '{station}'"
existingdf = pd.read_sql(sql, con=engine)
existingdf.head()



,stationid,datetime_start,timestamp_start,timestamp_end,co2,co2_sigma,h2o,h2o_sigma,fc,fc_ssitc_test,...,rh_1_3_1,t_dp_1_3_1,ta_1_4_1,pblh_f,ts_2_1_1,swc_2_1_1,t_canopy,ch4,fch4,fch4_ssitc_test
0,US-UTJ,2024-01-20 03:00:00,202401200300,202401200330,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None,None,None,None
1,US-UTJ,2024-01-20 03:30:00,202401200330,202401200400,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None,None,None,None
2,US-UTJ,2024-01-20 04:00:00,202401200400,202401200430,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None,None,None,None
3,US-UTJ,2024-01-20 04:30:00,202401200430,202401200500,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None,None,None,None
4,US-UTJ,2024-01-20 05:00:00,202401200500,202401200530,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None,None,None,None


In [23]:
import pandas as pd

# SQL query to get the max value from a field
query = f"SELECT MAX(timestamp_end) AS max_value FROM amfluxeddy WHERE stationid = '{station}';"

# Execute query and fetch results into a DataFrame
df = pd.read_sql(query, con=engine)

# Access the max value
max_value = df['max_value'].iloc[0]
print("Max value:", max_value)

Max value: 202410081500


In [20]:
df

,max_value
0,None


In [6]:
site_folders = {#'US-UTD':'Dugout_Ranch',
                #'US-UTB':'BSF',
                'US-UTJ':'Bluff',
                #'US-UTW':'Wellington',
                #'US-UTE':'Escalante',
                #'US-UTM':'Matheson',
                #'US-UTP':'Phrag',
                #'US-CdM':'Cedar_Mesa',
                #'US-UTV':'Desert_View_Myton',
                #'US-UTL':'Pelican_Lake',
                }

#stations = ['UTJ','UTV','UTP','UTW','UTM','CdM','UTL','UTE']
compdf = {}
for stationid,name in site_folders.items():
    station = stationid.split('-')[-1]
    if 'eddy' in config[station].keys():
        stationtime, comptime = micromet.get_times(station,config,loggertype='eddy')
        am_df, pack_size = micromet.get_station_data(station,config,loggertype='eddy')

        if am_df is None:
            mindate = am_df['TIMESTAMP_START'].min()
            maxdate = am_df['TIMESTAMP_START'].min()
        else:
            mindate = None
            maxdate = None

        am_df.to_postgres('amfluxeddy',con=engine, if_exists='append', index=False)

    elif 'met' in config[station].keys():
        stationtime, comptime = micromet.get_times(station,config,loggertype='met')
        am_df, pack_size = micromet.get_station_data(station,config,loggertype='met')

        if am_df is None:
            mindate = am_df['TIMESTAMP_START'].min()
            maxdate = am_df['TIMESTAMP_START'].min()
        else:
            mindate = None
            maxdate = None

        am_df.to_postgres('amfluxmet',con=engine, if_exists='append', index=False)

            #micromet.outfile(am_df, stationid, amflux_out)




UTJ


In [14]:
len(response_1.content)*1E-6

3.783502

In [9]:
mindate = am_df['TIMESTAMP_START'].min()
maxdate = am_df['TIMESTAMP_START'].min()

np.int64(202501160830)

In [5]:
#http://192.168.4.14/?command=ClockCheck
station = 'UTJ'
ip = config[station]['ip']
clk_url = f"http://{ip}:{port}/?"
#url = f"http://{ip}/tables.html?command=DataQuery&mode=since-record&format=toA5&uri=dl:Flux_AmeriFluxFormat&p1=0"
clk_args = {'command':'ClockCheck',
        'uri':'dl',
        'format':'json',
        }
clktimeresp = requests.get(clk_url, params=clk_args, auth=HTTPBasicAuth(config['LOGGER']['login'], config['LOGGER']['pw'])).json()
if 'time' in clktimeresp.keys():
    clktime = clktimeresp['time']
else:
    clktime = None

comptime = f"{datetime.datetime.now():%Y-%m-%d %H:%M:%S}"

In [7]:
comptime

'2025-01-23 12:25:01'

In [11]:
station = 'UTE'
ip = config[station]['ip']
port = config[station]['met']
url = f"http://{ip}:{port}/tables.html?"
params = {'command':'DataQuery',
          'mode':'since-record',
          'format':'toA5',
          'uri':'dl:Statistics_AmeriFlux',
          'p1':0,
          }
response_1 = requests.get(url, params=params ,auth=HTTPBasicAuth(config['LOGGER']['login'], config['LOGGER']['pw']))
if response_1.status_code == 200:
    headers = pd.read_csv(BytesIO(response_1.content),skiprows=[0]).iloc[0:2,:].T
    raw_data = pd.read_csv(BytesIO(response_1.content),skiprows=[0,2,3])



In [12]:
raw_data

,TIMESTAMP,RECORD,TIMESTAMP_START,TIMESTAMP_END,ALB,NETRAD,SW_IN,SW_OUT,LW_IN,LW_OUT,...,WS,WD,LWmV_1_1_1,LWMDry_1_1_1,LWMCon_1_1_1,LWMWet_1_1_1,LWmV_1_1_2,LWMDry_1_1_2,LWMCon_1_1_2,LWMWet_1_1_2
0,2024-03-12 09:30:00,0,202403120919,202403120930,18.49048,-21.39694,30.728240,5.684656,401.7438,448.1842,...,9.532032,314.95310,258.8016,36.38889,0.000000,0.00000,258.5672,36.38889,0.00000,0.00000
1,2024-03-12 10:00:00,1,202403120930,202403121000,19.33819,-14.17144,35.109390,6.790694,402.3058,444.7959,...,7.415333,334.09340,259.4499,100.00000,0.000000,0.00000,259.2029,100.00000,0.00000,0.00000
2,2024-03-12 10:30:00,2,202403121000,202403121030,15.04910,15.07617,46.988440,7.859250,399.5011,423.5542,...,3.388243,319.68610,287.1852,41.38889,7.777778,50.83333,272.3126,62.50000,14.72222,22.77778
3,2024-03-12 11:00:00,3,202403121030,202403121100,23.07624,58.11319,123.988300,28.730390,388.8729,426.0176,...,5.368657,317.82870,263.7308,84.72222,7.777778,7.50000,260.6151,100.00000,0.00000,0.00000
4,2024-03-12 11:30:00,4,202403121100,202403121130,24.28172,63.82758,147.530400,35.854500,382.3825,430.2309,...,4.270786,298.74190,258.9477,100.00000,0.000000,0.00000,260.2029,100.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8040,2025-01-23 02:30:00,8040,202501230200,202501230230,0.00000,-78.22307,-1.007444,-0.107667,177.3601,254.6834,...,1.642534,148.68750,263.5028,100.00000,0.000000,0.00000,263.1570,100.00000,0.00000,0.00000
8041,2025-01-23 03:00:00,8041,202501230230,202501230300,0.00000,-74.94375,-0.940555,-0.186000,175.9579,250.1471,...,1.154821,48.70794,263.5890,100.00000,0.000000,0.00000,263.3429,100.00000,0.00000,0.00000
8042,2025-01-23 03:30:00,8042,202501230300,202501230330,0.00000,-73.59842,-0.973944,-0.162778,174.6615,247.4488,...,1.414738,212.45710,263.9224,100.00000,0.000000,0.00000,263.5649,100.00000,0.00000,0.00000
8043,2025-01-23 04:00:00,8043,202501230330,202501230400,0.00000,-71.05519,-0.898167,-0.135306,172.6765,242.9689,...,1.306557,62.15498,264.2498,100.00000,0.000000,0.00000,263.8954,100.00000,0.00000,0.00000
